Building a lightweight document to question similarity evaluation; data too large to process with RNN|LSTM|Attention given current time and computational power available

In [75]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

df_master = pd.read_csv('master_data.csv')
del df_master['Unnamed: 0']

In [4]:
df_master.head()

,Unnamed: 0_x,book_id,question_id,section_id,Gutenberg,﻿,question,answer,tokens,what_ind,...,question_len,lexile_score,Unnamed: 0_y,context,tags,master_text,tokens_ans,question_c,answer_c,context_c
0,0,93,180,30,1,179.0,180. How many days after returning home\r\r\r\...,Two,"['180', '.', 'How', 'many', 'days', 'after', '...",False,...,12,1070,1619.0,\r\r\nIN WHICH IT IS SHOWN THAT PHILEAS FOGG G...,93\30.txt,180. how many days after returning home\r\r\r\...,['two'],180. how many days after returning home\r\r\r\...,two,\r\r\nin which it is shown that phileas fogg g...
1,1,93,179,30,1,178.0,179. What does Fogg get from his journey\r\r\r...,All of these,"['179', '.', 'What', 'does', 'Fogg', 'get', 'f...",True,...,11,1070,1619.0,\r\r\nIN WHICH IT IS SHOWN THAT PHILEAS FOGG G...,93\30.txt,179. what does fogg get from his journey\r\r\r...,"['all', 'of', 'these']",179. what does fogg get from his journey\r\r\r...,all of these,\r\r\nin which it is shown that phileas fogg g...
2,2,93,178,30,1,177.0,178. Who says That we might have\r\r\r\r\r\nma...,Passepartout,"['178.', 'Who', 'says', 'That', 'we', 'might',...",False,...,17,1070,1619.0,\r\r\nIN WHICH IT IS SHOWN THAT PHILEAS FOGG G...,93\30.txt,178. who says that we might have\r\r\r\r\r\nma...,['passepartout'],178. who says that we might have\r\r\r\r\r\nma...,passepartout,\r\r\nin which it is shown that phileas fogg g...
3,3,93,177,30,1,176.0,177. What does Passepartout do with the\r\r\r\...,He splits it between Passepartout and Fix,"['177', '.', 'What', 'does', 'Passepartout', '...",True,...,12,1070,1619.0,\r\r\nIN WHICH IT IS SHOWN THAT PHILEAS FOGG G...,93\30.txt,177. what does passepartout do with the\r\r\r\...,"['he', 'splits', 'it', 'between', 'passepartou...",177. what does passepartout do with the\r\r\r\...,he splits it between passepartout and fix,\r\r\nin which it is shown that phileas fogg g...
4,4,93,176,30,1,175.0,176. What does Passepartout discover on\r\r\r\...,They arrived on Saturday not Sunday,"['176', '.', 'What', 'does', 'Passepartout', '...",True,...,8,1070,1619.0,\r\r\nIN WHICH IT IS SHOWN THAT PHILEAS FOGG G...,93\30.txt,176. what does passepartout discover on\r\r\r\...,"['they', 'arrived', 'on', 'saturday', 'not', '...",176. what does passepartout discover on\r\r\r\...,they arrived on saturday not sunday,\r\r\nin which it is shown that phileas fogg g...


In [16]:
np.max(df_master['question_len'])

56

In [140]:
df_master['book_id'].unique()

array([ 93, 178, 175, 174, 173, 172, 171, 168, 167, 166, 165, 164, 163,
       162, 161, 160, 158, 157, 156, 155, 154, 153, 152, 150, 151, 148,
       147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135,
       134, 133, 132, 131, 130, 128,  35,  12,  79,  77,  83,  84,  85,
        86,  87,  88,  90,  89, 127,  91,  92,  94,  95,  96,  98, 100,
       101, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126],
      dtype=int64)

In [180]:
df_master.iloc[10530,]

Unnamed: 0_x                                                 10614
book_id                                                        127
question_id                                                    180
section_id                                                       5
Gutenberg                                                        1
﻿                                                             2879
question         180. What does the narrator say Thnardier goes...
answer                                              A slave-trader
tokens           ['180', '.', 'What', 'does', 'the', 'narrator'...
what_ind                                                      True
tokens2          ['', '.', 'what', 'does', 'the', 'narrator', '...
how_ind                                                      False
where_ind                                                    False
when_ind                                                     False
why_ind                                                      F

In [151]:
temp = df_master.loc[df_master.book_id == 89] 
temp.loc [df_master.question_id == 49] #df.loc[(df.Product == p_id)

,Unnamed: 0_x,book_id,question_id,section_id,Gutenberg,﻿,question,answer,tokens,what_ind,...,question_len,lexile_score,Unnamed: 0_y,context,tags,master_text,tokens_ans,question_c,answer_c,context_c
10488,10488,89,49,9,1,948.0,49. Why is Mr. Bumble planning to go to\r\r\r\...,To transfer two sick children to a\r\r\r\r\r\n...,"['49', '.', 'Why', 'is', 'Mr.', 'Bumble', 'pla...",False,...,20,1060,1529.0,CHAPTER XIX\r\r\n\r\r\nIN WHICH A NOTABLE PLAN...,89\9.txt,49. why is mr. bumble planning to go to\r\r\r\...,"['to', 'transfer', 'two', 'sick', 'children', ...",49. why is mr. bumble planning to go to\r\r\r\...,to transfer two sick children to a\r\r\r\r\r\n...,chapter xix\r\r\n\r\r\nin which a notable plan...


In [76]:
#Removing problem children:ex 'there is no chapter 25'
df_master = df_master.drop(df_master.index[[7020, 7021, 7022, 7023,7024, 7025, 7026,7027,7028,7029,7030, 7031,7032,7033,7034,
                                       7035,7036,7037,7038,7039,7040,7041,7042,7043]])#[8790,8791,8792,8793,8794 ,8795]]) #,8766,8767,8768,8769,8770,8771]])
#,10524, 10525,10526,10527,10528,10529]]) # THROUGH 7020

In [79]:
df_master = df_master.drop(df_master.index[[8790,8791,8792,8793,8794 ,8795]])

In [95]:
df_master = df_master.drop(df_master.index[[10578,10579,10580,10581,10582,10583]])

In [96]:
df_master.iloc[10584,] #13347,] #13354,]

Unnamed: 0_x                                                 10620
book_id                                                        127
question_id                                                    174
section_id                                                       5
Gutenberg                                                        1
﻿                                                             2873
question         174. What is the title of Book VIII in Part V ...
answer                                         The Twilight Wanes.
tokens           ['174', '.', 'What', 'is', 'the', 'title', 'of...
what_ind                                                      True
tokens2          ['', '.', 'what', 'is', 'the', 'title', 'of', ...
how_ind                                                      False
where_ind                                                    False
when_ind                                                     False
why_ind                                                      F

###### df_master.iloc[8819, ]

# Checking out some starter code

In [55]:
#check demo code
vect = TfidfVectorizer(min_df=1)
tfidf = vect.fit_transform(["I'd like an apple",
                             "An apple a day keeps the doctor away",
                             "Never compare an apple to an orange",
                             "I prefer scikit-learn to Orange"])
matrix_test = (tfidf * tfidf.T).A
matrix_test

array([[1.        , 0.25082859, 0.39482963, 0.        ],
       [0.25082859, 1.        , 0.22057609, 0.        ],
       [0.39482963, 0.22057609, 1.        , 0.26264139],
       [0.        , 0.        , 0.26264139, 1.        ]])

In [56]:
matrix_test = np.array(matrix_test)
matrix_test.argsort()

array([[3, 1, 2, 0],
       [3, 2, 0, 1],
       [1, 3, 0, 2],
       [0, 1, 2, 3]], dtype=int64)

In [17]:
test = df_master.iloc[0,26]
print(test)

test2 = df_master.iloc[1,26]


are fogg and aouda married


In [57]:
#test small sample
vect = TfidfVectorizer(min_df=1)
tfidf = vect.fit_transform([test,
                             test2])
matrix_test = (tfidf * tfidf.T).A
matrix_test

array([[1.        , 0.04604361],
       [0.04604361, 1.        ]])

In [58]:
matrix_test[0,1]

0.04604360647426782

In [4]:
context_test = df_master.iloc[1,28]
context_test

'\r\r\nin which it is shown that phileas fogg gained nothing by his tour\r\r\naround the world unless it were happiness\r\r\n\r\r\n\r\r\nyes phileas fogg in person.\r\r\n\r\r\nthe reader will remember that at five minutes past eight in the\r\r\nevening--about five and twenty hours after the arrival of the\r\r\ntravellers in london--passepartout had been sent by his master to\r\r\nengage the services of the reverend samuel wilson in a certain marriage\r\r\nceremony which was to take place the next day.\r\r\n\r\r\npassepartout went on his errand enchanted.  he soon reached the\r\r\nclergymans house but found him not at home.  passepartout waited a\r\r\ngood twenty minutes and when he left the reverend gentleman it was\r\r\nthirty-five minutes past eight.  but in what a state he was  with his\r\r\nhair in disorder and without his hat he ran along the street as never\r\r\nman was seen to run before overturning passers-by rushing over the\r\r\nsidewalk like a waterspout.\r\r\n\r\r\nin three

In [5]:
# code from stackoverflow https://stackoverflow.com/questions/4576077/python-split-text-on-sentences
# -*- coding: utf-8 -*-
import re
caps = "([A-Z])"
prefixes = "(mr|mrs|ms|dr|no)[.]"#added no, took 'st' out (exist., west. had problems and st. is less common
suffixes = "(inc|ltd|jr|sr|co)"
starters = "(mr|mrs|ms|mr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    if isinstance(text,float):
        text = "JUNK THAT SHOULD BE QC"
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = text.replace("\r"," ")
    text = text.replace("      ", " ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "ph.d" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".","<stop>")# took out punct
    text = text.replace("?","<stop>")# took out punct
    text = text.replace("!","<stop>")# took out punct
    text = text.replace("<prd>",".")# took out punct
    #I added to debug LSTM
    text = text.replace("-", " ")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip(r'\-=~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?]') for s in sentences]
    #sentences = sentences.replace('[`\-=~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?]',' ')
    if not sentences:
        sentences =  list(["JUNK", "THAT SHOULD", "BE removed", "when I have time","opps my bad"])
    return sentences
#print(context_test)
context_test2 = split_into_sentences(context_test)

In [322]:
split_into_sentences(df_master.iloc[8815,28])

['JUNK', 'THAT SHOULD', 'BE removed', 'when I have time', 'opps my bad']

In [317]:
split_into_sentences(df_master.iloc[2,28])

['    in which it is shown that phileas fogg gained nothing by his tour   around the world unless it were happiness    yes phileas fogg in person',
 ' the reader will remember that at five minutes past eight in the   evening  about five and twenty hours after the arrival of the   travellers in london  passepartout had been sent by his master to   engage the services of the reverend samuel wilson in a certain marriage   ceremony which was to take place the next day',
 ' passepartout went on his errand enchanted',
 '  he soon reached the   clergymans house but found him not at home',
 '  passepartout waited a   good twenty minutes and when he left the reverend gentleman it was   thirty five minutes past eight',
 '  but in what a state he was  with his   hair in disorder and without his hat he ran along the street as never   man was seen to run before overturning passers by rushing over the   sidewalk like a waterspout',
 ' in three minutes he was in saville row again and staggered back i

In [13]:
context_test2[5]

'  but in what a state he was  with his   hair in disorder and without his hat he ran along the street as never   man was seen to run before overturning passers by rushing over the   sidewalk like a waterspout'

In [124]:
context_test2[7].replace("-", " ")

'\r\r sir francis cromarty had observed the oddity of his travelling\r\r companion  although the only opportunity he had for studying him had\r\r been while he was dealing the cards and between two rubbers  and\r\r questioned himself whether a human heart really beat beneath this cold\r\r exterior and whether phileas fogg had any sense of the beauties of\r\r nature'

# Method 1: Compares the question to all other sentences in a set of chapters at once
Mediocre performance on the test example as evidenced by the correct answer being 6th on the distance list, but I thought it might do better because the tfidf would weed out more words. 

In [156]:
top_sent = []
second_best = []
third_best = []
fourth_best = []
fifth_best = []
similiarity_lst = []
#split up chapters into sentences
context = split_into_sentences(context_test) 
context.append(test)
#compare each sentence with the question and calc similarity
#for sentence in range(0, len(context)):
    #compare question and sentence
vect = TfidfVectorizer(min_df=1)
tfidf = vect.fit_transform(context)
cos_matrix = (tfidf * tfidf.T).A
cos_matrix

array([[1.        , 0.16587784, 0.03040895, ..., 0.        , 0.02963158,
        0.03551618],
       [0.16587784, 1.        , 0.04510202, ..., 0.06270105, 0.11354278,
        0.04518333],
       [0.03040895, 0.04510202, 1.        , ..., 0.        , 0.03917223,
        0.        ],
       ...,
       [0.        , 0.06270105, 0.        , ..., 1.        , 0.05238752,
        0.        ],
       [0.02963158, 0.11354278, 0.03917223, ..., 0.05238752, 1.        ,
        0.01537483],
       [0.03551618, 0.04518333, 0.        , ..., 0.        , 0.01537483,
        1.        ]])

In [121]:
context

['in which it is shown that phileas fogg gained nothing by his tour\r\r around the world unless it were happiness\r\r \r\r \r\r yes phileas fogg in person.',
 'the reader will remember that at five minutes past eight in the\r\r evening--about five and twenty hours after the arrival of the\r\r travellers in london--passepartout had been sent by his master to\r\r engage the services of the reverend samuel wilson in a certain marriage\r\r ceremony which was to take place the next day.',
 'passepartout went on his errand enchanted.',
 'he soon reached the\r\r clergymans house but found him not at home.',
 'passepartout waited a\r\r good twenty minutes and when he left the reverend gentleman it was\r\r thirty-five minutes past eight.',
 'but in what a state he was  with his\r\r hair in disorder and without his hat he ran along the street as never\r\r man was seen to run before overturning passers-by rushing over the\r\r sidewalk like a waterspout.',
 'in three minutes he was in saville row 

In [157]:
temp = np.array(cos_matrix)[-1]#[:][::-1]

In [158]:
temp.argsort()[-10:]

array([27, 35, 26, 31, 36,  3, 16, 18, 30, 39], dtype=int64)

In [159]:
context[39]

'180. how many days after returning home\r\r\r\r\r\nare fogg and aouda married'

In [160]:
context[30]

'dear aouda replied phileas fogg.'

In [161]:
context[18]

'in journeying eastward he had gone towards the sun and the days\r\r therefore diminished for him as many times four minutes as he crossed\r\r degrees in this direction.'

In [162]:
context[16]

'phileas fogg had accomplished the journey round the world in eighty\r\r days\r\r \r\r phileas fogg had won his wager of twenty thousand pounds\r\r \r\r how was it that a man so exact and fastidious could have made this\r\r error of a day  how came he to think that he had arrived in london on\r\r saturday the twenty-first day of december when it was really friday\r\r the twentieth the seventy-ninth day only from his departure\r\r \r\r the cause of the error is very simple.'

In [163]:
context[3]

'he soon reached the\r\r clergymans house but found him not at home.'

In [164]:
context[36]

'phileas fogg had won his wager and had made his journey around the\r\r world in eighty days.'

In [165]:
context[31]

'it need not be said that the marriage took place forty-eight hours\r\r after and that passepartout glowing and dazzling gave the bride\r\r away.'

Row 31 contains the answer so this method did not perform spectacuarly because the 6th sentence might not make the cut into the model

# Method 2: compares only 1 sentence to the question
Actually did much better on the test example (correct answer #4 best score).

In [118]:
top_sent = []
second_best = []
third_best = []
fourth_best = []
fifth_best = []
similiarity_lst = []
#split up chapters into sentences
context = split_into_sentences(context_test)
#compare each sentence with the question and calc similarity
for sentence in range(0, len(context)):
    #compare question and sentence
    vect = TfidfVectorizer(min_df=1)
    tfidf = vect.fit_transform([test, context[sentence]])
    cos_matrix = (tfidf * tfidf.T).A
    #find best alignment score
    similiarity_lst.append(cos_matrix[0,1])
position_top5 = np.array(similiarity_lst).argsort()[-5:][::-1]
#store top values (didn't go with df or dict because it caused great pain trying to work into future LSTM code before)
for top5_sent in range(0,4):
    print (top5_sent)
    cont_index = position_top5[top5_sent]
    if top5_sent == 0: 
        top_sent.append(context[cont_index])
    if top5_sent == 1: 
        second_best.append(context[cont_index])
    if top5_sent == 2: 
        third_best.append(context[cont_index])
    if top5_sent == 3: 
        fourth_best.append(context[cont_index])
    if top5_sent == 4: 
        fifth_best.append(context[cont_index])


0
1
2
3


In [101]:
top_sent

['dear aouda replied phileas fogg.']

In [102]:
second_best

['phileas fogg had won his wager and had made his journey around the\r\r world in eighty days.']

Testing approach on the first question & context. The answer is found in the sentence with the 4th highest similarity score

In [71]:
similiarity_lst

[0.05358797469131102,
 0.029493743944264034,
 0.0,
 0.043997326267831816,
 0.032957177740365697,
 0.02188454095615924,
 0.03566946421355304,
 0.0,
 0.05850970413367294,
 0.0,
 0.04604360647426782,
 0.0,
 0.0,
 0.048847834143217424,
 0.07188587407390887,
 0.0,
 0.075126382581545,
 0.04076661992479196,
 0.07970251181266944,
 0.08456971633273137,
 0.022370855988075977,
 0.07188864279873158,
 0.061212946253748325,
 0.0406093257697673,
 0.030782327506493744,
 0.024369200829988422,
 0.09303603605128495,
 0.04604360647426782,
 0.0,
 0.04547582924081301,
 0.15231415519375907,
 0.08722192510054794,
 0.024700713493349607,
 0.0694071716911208,
 0.051171029771728015,
 0.08340941143953931,
 0.10599213135093252,
 0.0,
 0.0406093257697673]

In [88]:
print(sorted(similiarity_lst)[-5:])
print(similiarity_lst.index(max(similiarity_lst)))

[0.08456971633273137, 0.08722192510054794, 0.09303603605128495, 0.10599213135093252, 0.15231415519375907]
30


In [90]:
np.array(similiarity_lst).argsort()[-5:][::-1]

array([30, 36, 26, 31, 19], dtype=int64)

In [73]:
test

'180. how many days after returning home\r\r\r\r\r\nare fogg and aouda married'

In [77]:
context[30]

'dear aouda replied phileas fogg.'

In [86]:
context[36]

'phileas fogg had won his wager and had made his journey around the\r\r world in eighty days.'

In [87]:
context[26]

'that evening mr. fogg as tranquil and phlegmatic as ever said to\r\r aouda is our marriage still agreeable to you\r\r \r\r mr. fogg replied she it is for me to ask that question.'

In [91]:
context[31]

'it need not be said that the marriage took place forty-eight hours\r\r after and that passepartout glowing and dazzling gave the bride\r\r away.'

# Method 3: No tfidf & sentence to question
I began to wonder if in my 1:1 approach the overlapping words were important clues and being removed with tfidf, leaving the similiarity assessment to be based on extraneous words left behind. Just to prove to myself that countvect was not the way to go I ran method 3. The correct answer is no where in sight of the top sentences and the sentences that were selected were not remotely related.

In [139]:
from sklearn.feature_extraction.text import CountVectorizer

In [148]:
top_sent = []
second_best = []
third_best = []
fourth_best = []
fifth_best = []
similiarity_lst = []
#split up chapters into sentences
context = split_into_sentences(context_test)
#compare each sentence with the question and calc similarity
for sentence in range(0, len(context)):
    #compare question and sentence
    vect = CountVectorizer()
    tfidf = vect.fit_transform([test, context[sentence]])
    cos_matrix = (tfidf * tfidf.T).A
    #find best alignment score
    similiarity_lst.append(cos_matrix[0,1])
position_top5 = np.array(similiarity_lst).argsort()[-5:][::-1]
#store top values (didn't go with df or dict because it caused great pain trying to work into future LSTM code before)
for top5_sent in range(0,5):
    print (top5_sent)
    cont_index = position_top5[top5_sent]
    if top5_sent == 0: 
        top_sent.append(context[cont_index])
    if top5_sent == 1: 
        second_best.append(context[cont_index])
    if top5_sent == 2: 
        third_best.append(context[cont_index])
    if top5_sent == 3: 
        fourth_best.append(context[cont_index])
    if top5_sent == 4: 
        fifth_best.append(context[cont_index])


0
1
2
3
4


In [141]:
top_sent

['phileas fogg had accomplished the journey round the world in eighty\r\r days\r\r \r\r phileas fogg had won his wager of twenty thousand pounds\r\r \r\r how was it that a man so exact and fastidious could have made this\r\r error of a day  how came he to think that he had arrived in london on\r\r saturday the twenty-first day of december when it was really friday\r\r the twentieth the seventy-ninth day only from his departure\r\r \r\r the cause of the error is very simple.']

In [82]:
second_best

[]

In [151]:
third_best

['and passepartouts famous family watch which had always kept london\r\r time would have betrayed this fact if it had marked the days as well\r\r as the hours and the minutes\r\r \r\r phileas fogg then had won the twenty thousand pounds but as he had\r\r spent nearly nineteen thousand on the way the pecuniary gain was\r\r small.']

In [150]:
fourth_best

['that evening mr. fogg as tranquil and phlegmatic as ever said to\r\r aouda is our marriage still agreeable to you\r\r \r\r mr. fogg replied she it is for me to ask that question.']

In [149]:
fifth_best

['phileas fogg had won his wager and had made his journey around the\r\r world in eighty days.']

In [143]:
similiarity_lst

[2,
 2,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 2,
 3,
 0,
 6,
 2,
 3,
 4,
 1,
 2,
 4,
 1,
 1,
 1,
 4,
 1,
 0,
 2,
 2,
 3,
 1,
 3,
 1,
 3,
 3,
 0,
 1]


# Applying technique to the entire dataset

In [97]:
top_sent = []
similiarity_lst = []
dict_lstm = dict()
for row in range(0,len(df_master)):
    if not isinstance((df_master.iloc[row, 28]), str):
        dict_lstm[row] = ['JUNK']
        top_sent.append('unk')
        continue
    context_test = df_master.iloc[row, 28]
    if not isinstance((df_master.iloc[row, 26]), str):
        dict_lstm[row] = ['JUNK']
        top_sent.append('unk')
        question = 'Not a real question'
        #continue
    else: question = df_master.iloc[row,26]
    #split up chapters into sentences
    context = split_into_sentences(context_test) 
    context.append(question)
    #compare each sentence with the question and calc similarity
    vect = TfidfVectorizer(min_df=1)
    tfidf = vect.fit_transform(context)
    cos_matrix = (tfidf * tfidf.T).A
    cos_matrix
    #find top matches
    temp = np.array(cos_matrix)[-1]
    position_top5 = temp.argsort()[-10:]
    #store top values (didn't go with df or dict because it caused great pain trying to work into future LSTM code before)
    for top5_sent in range(0,len(position_top5)):
        cont_index = position_top5[top5_sent]
        if top5_sent == 0: 
            top_sent.append(context[cont_index])
    if not isinstance((df_master.iloc[row, 27]), str):
        dict_lstm[row] = ['JUNK']
        continue
    answer = df_master.iloc[row,27]
    #print(row)
    #print('top sent', top_sent)
    #print('top_sent indexed', top_sent[row])
    #data prep for LSTM
    dict_lstm[row] =  top_sent[row].split(), question.split(), answer.split()

In [377]:
 dict_lstm[7019]

(['it',
  'was',
  'completely',
  'covered',
  'with',
  'vines',
  'climbing',
  'roses',
  'and',
  'honeysuckles'],
 ['1.', 'which', 'genre', 'best', 'describes', 'this', 'novel'],
 ['autobiography'])

In [189]:
question.split()

['31.',
 'who',
 'makes',
 'the',
 'following',
 'statement',
 'in',
 'the',
 'way',
 'this',
 'strange',
 'gentleman',
 'was',
 'going',
 'on',
 'he',
 'would',
 'leave',
 'the',
 'world',
 'without',
 'having',
 'done',
 'any',
 'good',
 'to',
 'himself',
 'or',
 'anybody',
 'else.']

In [153]:
180-128

52

In [8]:
df_master.iloc[9870,]

Unnamed: 0_x                                                  9900
book_id                                                         87
question_id                                                      1
section_id                                                       1
Gutenberg                                                        1
﻿                                                             4221
question         1. What is the state of Jacob Marley at\r\r\r\...
answer                                                  He is dead
tokens           ['1', '.', 'What', 'is', 'the', 'state', 'of',...
what_ind                                                      True
tokens2          ['', '.', 'what', 'is', 'the', 'state', 'of', ...
how_ind                                                      False
where_ind                                                    False
when_ind                                                     False
why_ind                                                      F

In [98]:
#split dictionary into train and test
train_range = list(range(0,9870)) #10488  #10480,10492))
test_range = list(range(9870,12918 )) #10497))#9720,12989))

filterByKey = lambda keys: {x: dict_lstm[x] for x in keys}
dict_train = filterByKey(train_range)
dict_test = filterByKey(test_range)

#print(dict_train,dict_test) 

In [161]:
train_range

[10480,
 10481,
 10482,
 10483,
 10484,
 10485,
 10486,
 10487,
 10488,
 10489,
 10490,
 10491]

In [99]:
vocab = []
for key, value in dict_lstm.items():
    temp = [key,value]
    vocab.append(value)

In [135]:
vocab[170:180]

[(['id', 'like', 'to', 'see', 'you', 'do', 'it', 'in', 'eighty', 'days'],
  ['10.',
   'how',
   'much',
   'is',
   'wagered',
   'against',
   'fogg',
   'being',
   'able',
   'to',
   'travel',
   'the',
   'world',
   'in',
   '80',
   'days'],
  ['20000', 'pounds']),
 (['two',
   'first',
   'class',
   'tickets',
   'for',
   'paris',
   'having',
   'been',
   'speedily',
   'purchased',
   'mr.',
   'fogg',
   'was',
   'crossing',
   'the',
   'station',
   'to',
   'the',
   'train',
   'when',
   'he',
   'perceived',
   'his',
   'five',
   'friends',
   'of',
   'the',
   'reform'],
  ['9.',
   'what',
   'game',
   'does',
   'fogg',
   'enjoy',
   'playing',
   'at',
   'the',
   'reform',
   'club'],
  ['whist']),
 (['well',
   'ralph',
   'said',
   'thomas',
   'flanagan',
   'what',
   'about',
   'that',
   'robbery',
   'oh',
   'replied',
   'stuart',
   'the',
   'bank',
   'will',
   'lose',
   'the',
   'money'],
  ['8.', 'how', 'much', 'is', 'stolen', 'from',

In [100]:
vocab = [item for sublist in vocab for item in sublist]
vocab = [item for sublist in vocab for item in sublist]

In [ ]:
dict_lstm.values()

In [58]:
import time
start_time = time.time()

In [215]:
#vocab.append('cannot')
#vocab.append('streams.')
#vocab.append('vari')

In [62]:
end_time = time.time()
run_time = time.strftime("%H:%M:%S",time.gmtime(end_time - start_time))
print (' run time (H:M:S) ' + run_time)

 run time (H:M:S) 00:00:47


# Build LSTM Model

In [275]:
'''Trains a memory network on the Classics Comprehension dataset 
Original code taken from: https://github.com/keras-team/keras/blob/master/examples/babi_memnn.py
and modfied to fit a new dataset

References:

- Jason Weston, Antoine Bordes, Sumit Chopra, Tomas Mikolov, Alexander M. Rush,
  "Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks",
  http://arxiv.org/abs/1502.05698

- Sainbayar Sukhbaatar, Arthur Szlam, Jason Weston, Rob Fergus,
  "End-To-End Memory Networks",
  http://arxiv.org/abs/1503.08895
Reaches 98.6% accuracy on task 'single_supporting_fact_10k' after 120 epochs.
Time per epoch: 3s on CPU (core i7).
'''

start_time = time.time()

from __future__ import print_function
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re

def get_stories(f, only_supporting=False, max_length=None):

    '''Given the cleaned and tokenized dataset
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''

    data = f #parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data):

    inputs, queries, answers = [], [], []
    i = 0
    for story, query, answer in data:
        #print(data)
        #print('story view', story)
        #print('query view', query)
        #print (i)
        ##print('answer view', answer)
        i += 1
        inputs.append([word_idx[w] for w in story])
        queries.append([word_idx[w] for w in query])
        #if i == 1:
        answers.append(max([word_idx[w] for w in answer])) #to just do first word for now
        #print(answer)
    return (pad_sequences(inputs, maxlen=story_maxlen),
            pad_sequences(queries, maxlen=query_maxlen),
            np.array(answers))

#taken from csv input above & then passed through function to truncate the context to the appropriate length
train_stories = dict_train
test_stories = dict_test

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = 150 #max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = 40 #max(map(len, (x for _, x, _ in train_stories + test_stories)))
ans_maxlen = 1 #new addition, just to test code for now, #MVP

end_time = time.time()
run_time = time.strftime("%H:%M:%S",time.gmtime(end_time - start_time))
print ('Truncate run time (H:M:S) ' + run_time)

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')


start_time = time.time()

#print("jen's vocab final test", type(vocab))
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories.values())
inputs_test, queries_test, answers_test = vectorize_stories(test_stories.values())

print("Jen answers_test", answers_test)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')

end_time = time.time()
run_time = time.strftime("%H:%M:%S",time.gmtime(end_time - start_time))
print ('Vectorize run time (H:M:S) ' + run_time)

Truncate run time (H:M:S) 00:00:00
-
Vocab size: 778814 unique words
Story max length: 150 words
Query max length: 40 words
Number of training stories: 9870
Number of test stories: 3048
-
Here's what a "story" tuple looks like (input, query, answer):
(['you', 'were', 'ruined', 'but', 'now', 'you', 'are', 'rich', 'again'], ['180.', 'how', 'many', 'days', 'after', 'returning', 'home', 'are', 'fogg', 'and', 'aouda', 'married'], ['two'])
-
Vectorizing the word sequences...
Jen answers_test [778726 778656 769663 ... 778748 745129 776480]
-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (9870, 150)
inputs_test shape: (3048, 150)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (9870, 40)
queries_test shape: (3048, 40)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (9870,)
answers_test shape: (3048,)
-
Compiling...
Vectorize run time (H:M:S) 00:00:00


# Actually running Model here

In [103]:
start_time = time.time()
# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size, output_dim=64))
input_encoder_m.add(Dropout(0.3))

# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size, output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))

# output: (samples, story_maxlen, query_maxlen)



# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=query_maxlen))
question_encoder.add(Dropout(0.3))

# output: (samples, query_maxlen, embedding_dim)



# encode input sequence and questions (which are indices)

# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)


# compute a 'match' between the first input vector sequence and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.

# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)

# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train
model.fit([inputs_train, queries_train], answers_train, batch_size=32, epochs=10,
          validation_data=([inputs_test, queries_test], answers_test))
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

end_time = time.time()
run_time = time.strftime("%H:%M:%S",time.gmtime(end_time - start_time))
print ('Model run time (H:M:S) ' + run_time)

Train on 9870 samples, validate on 3048 samples
Epoch 1/10
9870/9870 [==============================] - 1043s 106ms/step - loss: 8.7056 - acc: 0.1882 - val_loss: 7.3833 - val_acc: 0.1608
Epoch 2/10
9870/9870 [==============================] - 1019s 103ms/step - loss: 6.3552 - acc: 0.1902 - val_loss: 7.5929 - val_acc: 0.1608
Epoch 3/10
9870/9870 [==============================] - 1021s 103ms/step - loss: 6.3291 - acc: 0.1902 - val_loss: 7.2121 - val_acc: 0.1608
Epoch 4/10
9870/9870 [==============================] - 1020s 103ms/step - loss: 6.1684 - acc: 0.1902 - val_loss: 7.2845 - val_acc: 0.1608
Epoch 5/10
9870/9870 [==============================] - 1019s 103ms/step - loss: 6.0651 - acc: 0.1902 - val_loss: 7.1983 - val_acc: 0.1608
Epoch 6/10
9870/9870 [==============================] - 1019s 103ms/step - loss: 5.9770 - acc: 0.1902 - val_loss: 7.1618 - val_acc: 0.1608
Epoch 7/10
9870/9870 [==============================] - 1020s 103ms/step - loss: 5.8870 - acc: 0.1903 - val_loss: 7.24

In [104]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model.1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5.1")
print("Saved model to disk")

Saved model to disk


In [191]:
predictions = model.predict([inputs_test, queries_test], answers_test)

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
[inputs_test, queries_test], answers_test

In [233]:
predictions = model.predict([inputs_test, queries_test]) 

InvalidArgumentError: indices[4,31] = 781481 is not in [0, 778814)
	 [[Node: sequential_3/embedding_3/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_3/embeddings/read, sequential_3/embedding_3/Cast)]]

Caused by op 'sequential_3/embedding_3/Gather', defined at:
  File "C:\Users\lucky\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\lucky\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\lucky\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\lucky\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\lucky\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 478, in start
    self.io_loop.start()
  File "C:\Users\lucky\Anaconda3\lib\site-packages\zmq\eventlo

In [ ]:
predictions = model.predict(

# Method 2 Applied to enitre dataset

In [105]:
top_sent2 = []
dict_lstm2 = dict()
for row in range(0,len(df_master)):
    print(row)
    similiarity_lst = []
    if not isinstance((df_master.iloc[row, 28]), str):
        dict_lstm2[row] = ['JUNK']
        top_sent2.append('unk')
        continue
    context_test = df_master.iloc[row, 28]
    if not isinstance((df_master.iloc[row, 26]), str):
        dict_lstm2[row] = ['JUNK']
        top_sent2.append('unk')
        question = 'Not a real question'
        #continue
    else: question = df_master.iloc[row,26]
    #split up chapters into sentences
    context = split_into_sentences(context_test)
    #compare each sentence with the question and calc similarity
    for sentence in range(0, len(context)):
        #compare question and sentence
        vect = TfidfVectorizer(min_df=1)
        tfidf = vect.fit_transform([question, context[sentence]])
        cos_matrix = (tfidf * tfidf.T).A
        #find best alignment score
        similiarity_lst.append(cos_matrix[0,1])
    position_top5 = np.array(similiarity_lst).argsort()[-5:][::-1]
    #store top values (didn't go with df or dict because it caused great pain trying to work into future LSTM code before)
    for top5_sent in range(0,2):
        #print (top5_sent)
        cont_index = position_top5[top5_sent]
        if top5_sent == 0: 
            top_sent2.append(context[cont_index])
    if not isinstance((df_master.iloc[row, 27]), str):
        dict_lstm2[row] = ['JUNK']
        continue
    answer = df_master.iloc[row,27]
    dict_lstm2[row] =  top_sent2[row].split(), question.split(), answer.split()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978


...

10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
10190
10191
10192
10193
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
10208
10209
10210
10211
10212
10213
1021

11414
11415
11416
11417
11418
11419
11420
11421
11422
11423
11424
11425
11426
11427
11428
11429
11430
11431
11432
11433
11434
11435
11436
11437
11438
11439
11440
11441
11442
11443
11444
11445
11446
11447
11448
11449
11450
11451
11452
11453
11454
11455
11456
11457
11458
11459
11460
11461
11462
11463
11464
11465
11466
11467
11468
11469
11470
11471
11472
11473
11474
11475
11476
11477
11478
11479
11480
11481
11482
11483
11484
11485
11486
11487
11488
11489
11490
11491
11492
11493
11494
11495
11496
11497
11498
11499
11500
11501
11502
11503
11504
11505
11506
11507
11508
11509
11510
11511
11512
11513
11514
11515
11516
11517
11518
11519
11520
11521
11522
11523
11524
11525
11526
11527
11528
11529
11530
11531
11532
11533
11534
11535
11536
11537
11538
11539
11540
11541
11542
11543
11544
11545
11546
11547
11548
11549
11550
11551
11552
11553
11554
11555
11556
11557
11558
11559
11560
11561
11562
11563
11564
11565
11566
11567
11568
11569
11570
11571
11572
11573
11574
11575
11576
11577
11578
11579
1158

12781
12782
12783
12784
12785
12786
12787
12788
12789
12790
12791
12792
12793
12794
12795
12796
12797
12798
12799
12800
12801
12802
12803
12804
12805
12806
12807
12808
12809
12810
12811
12812
12813
12814
12815
12816
12817
12818
12819
12820
12821
12822
12823
12824
12825
12826
12827
12828
12829
12830
12831
12832
12833
12834
12835
12836
12837
12838
12839
12840
12841
12842
12843
12844
12845
12846
12847
12848
12849
12850
12851
12852
12853
12854
12855
12856
12857
12858
12859
12860
12861
12862
12863
12864
12865
12866
12867
12868
12869
12870
12871
12872
12873
12874
12875
12876
12877
12878
12879
12880
12881
12882
12883
12884
12885
12886
12887
12888
12889
12890
12891
12892
12893
12894
12895
12896
12897
12898
12899
12900
12901
12902
12903
12904
12905
12906
12907
12908
12909
12910
12911
12912
12913
12914
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
12938
12939
12940
12941
12942
12943
12944
12945
12946
1294

14148
14149
14150
14151
14152
14153
14154
14155
14156
14157
14158
14159
14160
14161
14162
14163
14164
14165
14166
14167
14168
14169
14170
14171
14172
14173
14174
14175
14176
14177
14178
14179
14180
14181
14182
14183
14184
14185
14186
14187
14188
14189
14190
14191
14192
14193
14194
14195
14196
14197
14198
14199
14200
14201
14202
14203
14204
14205
14206
14207
14208
14209
14210
14211
14212
14213
14214
14215
14216
14217
14218
14219
14220
14221
14222
14223
14224
14225
14226
14227
14228
14229
14230
14231
14232
14233
14234
14235
14236
14237
14238
14239
14240
14241
14242
14243
14244
14245
14246
14247
14248
14249
14250
14251
14252
14253
14254
14255
14256
14257
14258
14259
14260
14261
14262
14263
14264
14265
14266
14267
14268
14269
14270
14271
14272
14273
14274
14275
14276
14277
14278
14279
14280
14281
14282
14283
14284
14285
14286
14287
14288
14289
14290
14291
14292
14293
14294
14295
14296
14297
14298
14299
14300
14301
14302
14303
14304
14305
14306
14307
14308
14309
14310
14311
14312
14313
1431

15514
15515
15516
15517
15518
15519
15520
15521
15522
15523
15524
15525
15526
15527
15528
15529
15530
15531
15532
15533
15534
15535
15536
15537
15538
15539
15540
15541
15542
15543
15544
15545
15546
15547
15548
15549
15550
15551
15552
15553
15554
15555
15556
15557
15558
15559
15560
15561
15562
15563
15564
15565
15566
15567
15568
15569
15570
15571
15572
15573
15574
15575
15576
15577
15578
15579
15580
15581
15582
15583
15584
15585
15586
15587
15588
15589
15590
15591
15592
15593
15594
15595
15596
15597
15598
15599
15600
15601
15602
15603
15604
15605
15606
15607
15608
15609
15610
15611
15612
15613
15614
15615
15616
15617
15618
15619
15620
15621
15622
15623
15624
15625
15626
15627
15628
15629
15630
15631
15632
15633
15634
15635
15636
15637
15638
15639
15640
15641
15642
15643
15644
15645
15646
15647
15648
15649
15650
15651
15652
15653
15654
15655
15656
15657
15658
15659
15660
15661
15662
15663
15664
15665
15666
15667
15668
15669
15670
15671
15672
15673
15674
15675
15676
15677
15678
15679
1568

In [106]:
#split dictionary into train and test
train_range = list(range(0,9870)) #10488  #10480,10492))
test_range = list(range(9870,12918 )) #10497))#9720,12989))

filterByKey = lambda keys: {x: dict_lstm2[x] for x in keys}
dict_train2 = filterByKey(train_range)
dict_test2 = filterByKey(test_range)

vocab2 = []
for key, value in dict_lstm2.items():
    temp = [key,value]
    vocab2.append(value)
    
vocab2 = [item for sublist in vocab2 for item in sublist]
vocab2 = [item for sublist in vocab2 for item in sublist]

In [8]:
df_master.iloc[12918,]

Unnamed: 0_x                                                 12954
book_id                                                        106
question_id                                                      1
section_id                                                       1
Gutenberg                                                        1
﻿                                                             1701
question                         1. Where does Aunt Polly find Tom
answer                                                 in a closet
tokens           ['1', '.', 'Where', 'does', 'Aunt', 'Polly', '...
what_ind                                                     False
tokens2          ['', '.', 'where', 'does', 'aunt', 'polly', 'f...
how_ind                                                      False
where_ind                                                     True
when_ind                                                     False
why_ind                                                      F

# Build Model Outline: LSTM method 2

In [13]:
from __future__ import print_function
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re

C:\Users\lucky\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [107]:

start_time = time.time()

def get_stories(f, only_supporting=False, max_length=None):

    '''Given the cleaned and tokenized dataset
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''

    data = f #parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data):

    inputs, queries, answers = [], [], []
    i = 0
    for story, query, answer in data:
        #print(data)
        #print('story view', story)
        print('query view', query)
        #print (i)
        ##print('answer view', answer)
        i += 1
        inputs.append([word_idx[w] for w in story])
        queries.append([word_idx[w] for w in query])
        #if i == 1:
        answers.append(max([word_idx[w] for w in answer])) #to just do first word for now
        #print(answer)
    return (pad_sequences(inputs, maxlen=story_maxlen),
            pad_sequences(queries, maxlen=query_maxlen),
            np.array(answers))

#taken from csv input above & then passed through function to truncate the context to the appropriate length
train_stories = dict_train2
test_stories = dict_test2

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab2) + 1
story_maxlen = 150 #max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = 40 #max(map(len, (x for _, x, _ in train_stories + test_stories)))
ans_maxlen = 1 #new addition, just to test code for now, #MVP

end_time = time.time()
run_time = time.strftime("%H:%M:%S",time.gmtime(end_time - start_time))
print ('Truncate run time (H:M:S) ' + run_time)

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')


start_time = time.time()

#print("jen's vocab final test", type(vocab))
word_idx = dict((c, i + 1) for i, c in enumerate(vocab2))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories.values())
inputs_test, queries_test, answers_test = vectorize_stories(test_stories.values())

print("Jen answers_test", answers_test)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')

end_time = time.time()
run_time = time.strftime("%H:%M:%S",time.gmtime(end_time - start_time))
print ('Vectorize run time (H:M:S) ' + run_time)

Truncate run time (H:M:S) 00:00:00
-
Vocab size: 781744 unique words
Story max length: 150 words
Query max length: 40 words
Number of training stories: 9870
Number of test stories: 3048
-
Here's what a "story" tuple looks like (input, query, answer):
(['dear', 'aouda', 'replied', 'phileas', 'fogg'], ['180.', 'how', 'many', 'days', 'after', 'returning', 'home', 'are', 'fogg', 'and', 'aouda', 'married'], ['two'])
-
Vectorizing the word sequences...
query view ['180.', 'how', 'many', 'days', 'after', 'returning', 'home', 'are', 'fogg', 'and', 'aouda', 'married']
query view ['179.', 'what', 'does', 'fogg', 'get', 'from', 'his', 'journey', 'around', 'the', 'world']
query view ['178.', 'who', 'says', 'that', 'we', 'might', 'have', 'made', 'the', 'tour', 'of', 'the', 'world', 'in', 'only', 'seventy-eight', 'days.']
query view ['177.', 'what', 'does', 'passepartout', 'do', 'with', 'the', 'remaining', 'money', 'from', 'his', 'journey']
query view ['176.', 'what', 'does', 'passepartout', 'discov

query view ['7.', 'what', 'did', 'solomon', 'learn', 'while', 'spending', 'time', 'in', 'the', 'yard', 'outside', 'williams', 'slave', 'pen']
query view ['6.', 'who', 'was', 'likely', 'to', 'be', 'behind', 'the', 'idea', 'of', 'jacob', 'brooks', 'taking', 'eliza', 'to', 'get', 'her', 'freedom', 'papers']
query view ['5.', 'what', 'was', 'the', 'purpose', 'of', 'having', 'jacob', 'brooks', 'accompany', 'eliza', 'to', 'get', 'her', 'freedom', 'papers']
query view ['4.', 'how', 'were', 'solomon', 'and', 'the', 'other', 'slaves', 'treated', 'around', 'goodins', 'pen', 'in', 'richmond', 'virginia']
query view ['3.', 'why', 'was', 'clemens', 'ray', 'upset', 'as', 'he', 'headed', 'south', 'aboard', 'the', 'orleans']
query view ['2.', 'how', 'can', 'the', 'manner', 'in', 'which', 'solomon', 'eliza', 'and', 'the', 'other', 'slaves', 'were', 'led', 'from', 'williams', 'slave', 'pen', 'and', 'onto', 'the', 'orleans', 'best', 'be', 'described']
query view ['1.', 'why', 'was', 'jacob', 'brooks', 'w

query view ['70.', 'the', 'woman', 'who', 'moves', 'in', 'with', 'fralein', 'brstner', 'holds', 'what', 'profession']
query view ['69.', 'who', 'moves', 'into', 'fralein', 'brstners', 'room', 'with', 'her', 'in', 'the', 'course', 'of', 'the', 'novel']
query view ['68.', 'what', 'word', 'from', 'in', 'the', 'empty', 'conference', 'hall', 'the', 'student', 'the', 'offices', 'refers', 'to', 'a', 'person', 'engaged', 'in', 'a', 'lawsuit']
query view ['67.', 'the', 'young', 'woman', 'in', 'the', 'court', 'offices', 'tells', 'k.', 'in', 'in', 'the', 'empty', 'conference', 'hall', 'the', 'student', 'the', 'offices', 'we', 'that', 'is', 'the', 'staff', 'here', 'decided', 'that', 'the', 'information', 'clerk', 'who', 'is', 'constantly', 'dealing', 'with', 'the', 'public', 'and', 'is', 'their', 'first', 'contact', 'with', 'the', 'court', 'ought', 'to', 'be', 'given', 'what']
query view ['66.', 'what', 'word', 'used', 'in', 'in', 'the', 'empty', 'conference', 'hall', 'the', 'student', 'the', 'off

query view ['42.', 'what', 'color', 'is', 'the', 'light', 'coming', 'through', 'the', 'windows']
query view ['41.', 'what', 'does', 'the', 'narrator', 'say', 'is', 'inaccessible', 'in', 'the', 'room', 'to', 'which', 'he', 'is', 'shown']
query view ['40.', 'what', 'is', 'the', 'expression', 'on', 'the', 'face', 'of', 'the', 'person', 'the', 'narrator', 'encounters', 'on', 'the', 'staircase']
query view ['39.', 'who', 'does', 'the', 'narrator', 'meet', 'on', 'one', 'of', 'the', 'staircases']
query view ['38.', 'what', 'happens', 'to', 'the', 'trophies', 'of', 'armor', 'when', 'the', 'narrator', 'walks', 'past', 'them']
query view ['37.', 'what', 'word', 'does', 'the', 'narrator', 'use', 'to', 'describe', 'the', 'tapestries']
query view ['36.', 'what', 'color', 'does', 'the', 'narrator', 'say', 'are', 'the', 'floors', 'in', 'the', 'house', 'of', 'usher']
query view ['35.', 'who', 'leads', 'the', 'narrator', 'through', 'the', 'house']
query view ['34.', 'where', 'is', 'the', 'narrator', 'f

query view ['27.', 'what', 'does', 'mr.', 'vincy', 'want', 'bulstrode', 'to', 'do']
query view ['26.', 'what', 'endeavor', 'is', 'the', 'banker', 'sponsoring']
query view ['25.', 'where', 'does', 'chapter', '13', 'take', 'place']
query view ['24.', 'what', 'is', 'bulstrodes', 'occupation']
query view ['23.', 'mary', 'garth', 'is', 'what', 'to', 'peter', 'featherstone']
query view ['22.', 'fred', 'vincy', 'is', 'what', 'relation', 'of', 'rosamond']
query view ['21.', 'whom', 'does', 'mr.', 'lydgate', 'admire']
query view ['20.', 'the', 'dinner', 'party', 'guests', 'question', 'mr.', 'brooke', 'about', 'his', 'endorsement', 'of', 'which', 'new', 'professional', 'in', 'town']
query view ['19.', 'what', 'was', 'casaubon', 'and', 'dorotheas', 'first', 'argument', 'about']
query view ['18.', 'what', 'does', 'will', 'want', 'to', 'do', 'in', 'the', 'future']
query view ['17.', 'will', 'ladislaw', 'is', 'what', 'relation', 'of', 'casaubon']
query view ['16.', 'dorothea', 'chooses', 'a', 'room'

...

query view ['32.', 'what', 'first', 'alerts', 'mrs.', 'loftus', 'to', 'the', 'fact', 'that', 'sarahmary', 'is', 'a', 'boy']
query view ['33.', 'what', 'news', 'does', 'mrs.', 'loftus', 'give', 'huck', 'regarding', 'his', 'murder']
query view ['34.', 'how', 'do', 'huck', 'and', 'jim', 'first', 'conceal', 'themselves', 'on', 'the', 'illinois', 'shore']
query view ['35.', 'how', 'does', 'huck', 'augment', 'his', 'and', 'jims', 'diet', 'of', 'river', 'fish', 'and', 'shot', 'waterfowl']
query view ['36.', 'how', 'do', 'huck', 'and', 'jim', 'enter', 'the', 'wrecked', 'ship']
query view ['37.', 'why', 'do', 'the', 'murderers', 'defer', 'leaving', 'in', 'the', 'skiff']
query view ['38.', 'what', 'is', 'the', 'name', 'of', 'the', 'sunken', 'steamer']
query view ['39.', 'what', 'convinces', 'the', 'ferry', 'captain', 'to', 'investigate', 'the', 'sinking', 'ship']
query view ['40.', 'in', 'chapter', '13', 'what', 'is', 'jims', 'attitude', 'to', 'the', 'adventure', 'he', 'and', 'huck', 'have', 'ju

query view ['132.', 'what', 'does', 'dickon', 'say', 'keeps', 'him', 'so', 'strong']
query view ['133.', 'how', 'long', 'does', 'collin', 'have', 'to', 'wait', 'to', 'see', 'the', 'garden', 'after', 'he', 'meets', 'dickon']
query view ['134.', 'what', 'is', 'collins', 'major', 'request', 'about', 'his', 'visits', 'to', 'the', 'garden']
query view ['135.', 'what', 'does', 'the', 'head', 'gardener', 'of', 'misselthwaite', 'think', 'of', 'dickon']
query view ['136.', 'what', 'does', 'collin', 'do', 'as', 'he', 'is', 'taken', 'into', 'the', 'garden']
query view ['137.', 'what', 'does', 'collin', 'say', 'once', 'he', 'is', 'in', 'the', 'garden']
query view ['138.', 'what', 'does', 'mrs.', 'medlock', 'tell', 'the', 'head', 'gardener', 'about', 'the', 'new', 'goings', 'on', 'in', 'the', 'house', 'just', 'before', 'collin', 'goes', 'out', 'for', 'the', 'first', 'time']
query view ['139.', 'what', 'does', 'collin', 'do', 'on', 'his', 'first', 'day', 'in', 'the', 'garden', 'while', 'mary', 'and'

query view ['92.', 'what', 'does', 'van', 'helsing', 'ask', 'from', 'seward', 'after', 'lucys', 'death']
query view ['93.', 'what', 'happens', 'to', 'holmwoods', 'father']
query view ['94.', 'who', 'is', 'lord', 'godalming']
query view ['95.', 'when', 'holmwood', 'looks', 'at', 'lucy', 'after', 'she', 'dies', 'what', 'does', 'he', 'notice']
query view ['96.', 'who', 'does', 'harker', 'think', 'he', 'sees', 'on', 'the', 'streets', 'of', 'london']
query view ['97.', 'when', 'does', 'mina', 'end', 'up', 'reading', 'harkers', 'diary']
query view ['98.', 'who', 'reads', 'all', 'of', 'lucys', 'diary', 'and', 'letters', 'in', 'an', 'attempt', 'to', 'understand', 'her', 'situation']
query view ['99.', 'why', 'does', 'van', 'helsing', 'want', 'mina', 'to', 'visit', 'after', 'lucys', 'death']
query view ['100.', 'what', 'kind', 'of', 'information', 'does', 'harker', 'agree', 'to', 'provide', 'for', 'van', 'helsing', 'after', 'lucys', 'death']
query view ['101.', 'how', 'does', 'mina', 'react', '

query view ['23.', 'what', 'hours', 'does', 'wilson', 'work', 'in', 'his', 'job', 'for', 'the', 'red-headed', 'league']
query view ['24.', 'what', 'is', 'wilson', 'instructed', 'to', 'copy', 'in', 'his', 'job', 'for', 'the', 'red-headed', 'league']
query view ['25.', 'after', 'holmes', 'accepts', 'wilsons', 'case', 'in', 'the', 'red-headed', 'league', 'what', 'does', 'he', 'do', 'to', 'help', 'him', 'think', 'about', 'the', 'case']
query view ['26.', 'in', 'the', 'red-headed', 'league', 'where', 'do', 'holmes', 'and', 'watson', 'go']
query view ['27.', 'what', 'body', 'part', 'of', 'spauldings', 'does', 'holmes', 'want', 'to', 'see']
query view ['28.', 'in', 'the', 'red-headed', 'league', 'what', 'does', 'holmes', 'ask', 'watson', 'to', 'bring', 'with', 'him', 'at', 'night']
query view ['29.', 'when', 'watson', 'shows', 'up', 'at', 'the', 'appointed', 'time', 'at', 'night', 'what', 'is', 'the', 'role', 'of', 'peter', 'jones']
query view ['30.', 'in', 'the', 'red-headed', 'league', 'bes

query view ['123.', 'who', 'is', 'sitting', 'in', 'the', 'front', 'row', 'during', 'darnays', 'trial']
query view ['124.', 'what', 'does', 'the', 'audience', 'yell', 'out', 'as', 'darnays', 'name', 'is', 'called']
query view ['125.', 'who', 'are', 'the', 'two', 'witnesses', 'darnay', 'names', 'in', 'his', 'behalf']
query view ['126.', 'what', 'is', 'the', 'verdict', 'rendered', 'to', 'charles', 'darnay']
query view ['127.', 'why', 'is', 'doctor', 'manette', 'sure', 'that', 'there', 'will', 'be', 'no', 'more', 'shoemaking', 'or', 'north', 'tower', 'for', 'him', 'ever', 'again']
query view ['128.', 'why', 'do', 'lucie', 'and', 'charles', 'not', 'hire', 'a', 'housekeeper']
query view ['129.', 'how', 'does', 'the', 'universal', 'fear', 'and', 'distrust', 'of', 'the', 'times', 'affect', 'daily', 'life', 'of', 'the', 'citizens']
query view ['130.', 'what', 'is', 'the', 'question', 'that', 'miss', 'pross', 'has', 'for', 'doctor', 'manette']
query view ['131.', 'what', 'startles', 'lucie', 'as

In [23]:
df_master.iloc[8784,]

Unnamed: 0_x                                                  8814
book_id                                                         12
question_id                                                    175
section_id                                                      25
Gutenberg                                                        1
﻿                                                             1338
question                               175. There is no chapter 25
answer                                 176. There is no chapter 25
tokens           ['175', '.', 'There', 'is', 'no', 'chapter', '...
what_ind                                                     False
tokens2              ['', '.', 'there', 'is', 'no', 'chapter', '']
how_ind                                                      False
where_ind                                                    False
when_ind                                                     False
why_ind                                                      F

In [22]:
dict_train2[8780]

(['where',
  'in',
  'the',
  'world',
  'did',
  'you',
  'come',
  'from',
  'from',
  'the',
  'land',
  'of',
  'oz',
  'said',
  'dorothy',
  'gravely'],
 ['171.', 'what', 'is', 'the', 'name', 'of', 'this', 'chapter'],
 ['home', 'again'])

In [108]:
start_time = time.time()
# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size, output_dim=64))
input_encoder_m.add(Dropout(0.3))

# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size, output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))

# output: (samples, story_maxlen, query_maxlen)



# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=query_maxlen))
question_encoder.add(Dropout(0.3))

# output: (samples, query_maxlen, embedding_dim)



# encode input sequence and questions (which are indices)

# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)


# compute a 'match' between the first input vector sequence and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.

# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)

# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model2 = Model([input_sequence, question], answer)
model2.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train
model2.fit([inputs_train, queries_train], answers_train, batch_size=32, epochs=10,
          validation_data=([inputs_test, queries_test], answers_test))
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

end_time = time.time()
run_time = time.strftime("%H:%M:%S",time.gmtime(end_time - start_time))
print ('Model run time (H:M:S) ' + run_time)

Train on 9870 samples, validate on 3048 samples
Epoch 1/10
9870/9870 [==============================] - 1026s 104ms/step - loss: 8.7446 - acc: 0.1927 - val_loss: 7.4101 - val_acc: 0.1627
Epoch 2/10
9870/9870 [==============================] - 1025s 104ms/step - loss: 6.2379 - acc: 0.1945 - val_loss: 7.2210 - val_acc: 0.1627
Epoch 3/10
9870/9870 [==============================] - 1024s 104ms/step - loss: 6.1128 - acc: 0.1945 - val_loss: 7.2878 - val_acc: 0.1627
Epoch 4/10
9870/9870 [==============================] - 1026s 104ms/step - loss: 6.0607 - acc: 0.1945 - val_loss: 7.1686 - val_acc: 0.1627
Epoch 5/10
9870/9870 [==============================] - 1025s 104ms/step - loss: 6.0062 - acc: 0.1945 - val_loss: 7.1540 - val_acc: 0.1627
Epoch 6/10
9870/9870 [==============================] - 1037s 105ms/step - loss: 5.9536 - acc: 0.1945 - val_loss: 7.2025 - val_acc: 0.1627
Epoch 7/10
9870/9870 [==============================] - 1027s 104ms/step - loss: 5.8794 - acc: 0.1944 - val_loss: 7.20

In [109]:
model2_json = model2.to_json()
with open("model2.json", "w") as json_file:
    json_file.write(model2_json)
# serialize weights to HDF5
model2.save_weights("model.h2")
print("Saved model to disk")

Saved model to disk


# BoW Comparison Model Method 1

In [262]:
#reformatting the data for BoW model
method1_test_data = [] 
method1_test_label = [] 
for passage, question, answer in dict_test.values():
    method1_test_data.append(' '.join(passage + question))
    method1_test_label.append(' '.join(answer))
df_m1_test_data = pd.DataFrame(method1_test_data)    
df_m1_test_label = pd.DataFrame(method1_test_label)

method1_train_data = [] 
method1_train_label = [] 
for passage, question, answer in dict_train.values():
    method1_train_data.append(' '.join(passage + question))
    method1_train_label.append(' '.join(answer))
df_m1_train_data = pd.DataFrame(method1_train_data)    
df_m1_train_label = pd.DataFrame(method1_train_label)

df_m1_train_data.fillna("", inplace=True)  
df_m1_test_data.fillna("", inplace=True) 

In [263]:
len(method1_test_data)
method1_test_data[1]
#method1_test_label[1]

'marleys ghost marley was dead to begin with 2. what is scrooges relationship to jacob marley'

In [264]:
df_m1_train_data = df_m1_train_data[0].astype(str)
print(df_m1_train_data.head()) #.astype(str)
df_m1_test_data = df_m1_test_data[0] #.astype(str)
print(df_m1_test_data.head())

0    you were ruined but now you are rich again 180...
1    the reader will remember that at five minutes ...
2    he deducted however from passepartouts share t...
3    passepartout went on his errand enchanted 177....
4    you have made a mistake of one day we arrived ...
Name: 0, dtype: object
0    is that the chance and hope you mentioned jaco...
1    marleys ghost marley was dead to begin with 2....
2    why did i walk through crowds of fellow beings...
3    mind i dont mean to say that i know of my own ...
4    you wish to be anonymous i wish to be left alo...
Name: 0, dtype: object


In [265]:
print('test data', df_m1_test_data.shape, 'test labels', df_m1_test_label.shape, 'train data', df_m1_train_data.shape, 'train labels', df_m1_train_label.shape)
#type(df_m2_test_data)

test data (3048,) test labels (3048, 1) train data (9870,) train labels (9870, 1)


In [266]:
df_m1_test_label = df_m1_test_label[0].ravel()
#df_m1_train_data = df_m1_train_data.ravel()
print('new test label shape', df_m1_test_label.shape, 'new train label shape', df_m1_train_label.shape)

new test label shape (3048,) new train label shape (9870, 1)


In [267]:
# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import * 
##################################################
vectorizer = TfidfVectorizer(lowercase=False) #already converted to lower case earlier, no need to run 2x
train_vec = vectorizer.fit_transform(df_m1_train_data)
test_vec = vectorizer.transform(df_m1_test_data)

print ("With the TFIDF leveraged, the vocabulary size shrinks to : " + str(train_vec.shape[1]) + "\n")

With the TFIDF leveraged, the vocabulary size shrinks to : 20730



In [268]:
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

start_time = time.time()

#Model with no preprocessing 
tfidf_LR_model_m1 = LogisticRegression(penalty="l2", C = 100)
tfidf_LR_model_m1.fit(train_vec, df_m1_train_label)

#makes predictions on the dev dataset with the model I just built
tfidf_dev_pred_labels_m1 = tfidf_LR_model_m1.predict(test_vec)
tfidf_dev_pred_probs_m1 = tfidf_LR_model_m1.predict_proba(test_vec)

from sklearn import metrics
metrics.f1_score(df_m1_test_label, tfidf_dev_pred_labels_m1, average='weighted')

end_time = time.time()
run_time = time.strftime("%H:%M:%S",time.gmtime(end_time - start_time))
print ('Model run time (H:M:S) ' + run_time)

C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model run time (H:M:S) 00:12:22


C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [269]:
metrics.f1_score(df_m1_test_label, tfidf_dev_pred_labels_m1, average='weighted')

C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.0017450405390158335

# BoW Comparison Model Method 2

In [123]:
#split dictionary into train and test
train_range = list(range(0,9870)) #10488  #10480,10492))
test_range = list(range(9870,12918 )) #10497))#9720,12989))

In [245]:
#reformatting the data for BoW model
method2_test_data = [] 
method2_test_label = [] 
for passage, question, answer in dict_test2.values():
    method2_test_data.append(' '.join(passage + question))
    method2_test_label.append(' '.join(answer))
df_m2_test_data = pd.DataFrame(method2_test_data)    
df_m2_test_label = pd.DataFrame(method2_test_label)

method2_train_data = [] 
method2_train_label = [] 
for passage, question, answer in dict_train2.values():
    method2_train_data.append(' '.join(passage + question))
    method2_train_label.append(' '.join(answer))
df_m2_train_data = pd.DataFrame(method2_train_data)    
df_m2_train_label = pd.DataFrame(method2_train_label)

df_m2_train_data.fillna("", inplace=True)  
df_m2_test_data.fillna("", inplace=True) 

In [248]:
len(method2_test_data)
method2_test_data[1]
#method2_test_label[1]

'old jacob marley tell me more speak comfort to me jacob i have none to give the ghost replied 2. what is scrooges relationship to jacob marley'

In [225]:
df_m2_train_data = df_m2_train_data[0].astype(str)
df_m2_train_data.head() #.astype(str)

0    dear aouda replied phileas fogg 180. how many ...
1    phileas fogg had won his wager and had made hi...
2    mr. fogg opened it and asked whats the matter ...
3    the reader will remember that at five minutes ...
4    passepartout went on his errand enchanted 176....
Name: 0, dtype: object

In [226]:
df_m2_test_data = df_m2_test_data[0] #.astype(str)
df_m2_test_data.head()

0       and being from the emotion he had undergone or...
1       old jacob marley tell me more speak comfort to...
2       scrooge stopped 3. which word best describes s...
3       expect the first to morrow when the bell tolls...
4       scrooge was not a man to be frightened by echo...
5       scrooge followed to the window desperate in hi...
6                    it is 7. what is scrooges first name
7       he went the whole length of the expression and...
8       but he couldnt replenish it for scrooge kept t...
9       but i am sure i have always thought of christm...
10      on the wings of the wind replied the ghost 11....
11      to edge his way along the crowded paths of lif...
12      but how much greater was his horror when the p...
13      scrooge followed to the window desperate in hi...
14      and being from the emotion he had undergone or...
15      it is also a fact that scrooge had seen it nig...
16      it is also a fact that scrooge had seen it nig...
17      mind i

In [227]:
print('test data', df_m2_test_data.shape, 'test labels', df_m2_test_label.shape, 'train data', df_m2_train_data.shape, 'train labels', df_m2_train_label.shape)
#type(df_m2_test_data)

test data (3048,) test labels (3048, 1) train data (9870,) train labels (9870, 1)


In [228]:
# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import * 
##################################################
#create n grams
#default CountVectorizer options 
#singleGram 
vectorizer = TfidfVectorizer(lowercase=False) #already converted to lower case earlier, no need to run 2x
train_vec = vectorizer.fit_transform(df_m2_train_data)
#train_vec.shape
#train_array = train_vec.toarray()
#print train_array [1:20,]
test_vec = vectorizer.transform(df_m2_test_data)
#print test_vec.shape

In [230]:
print ("With the TFIDF leveraged, the vocabulary size shrinks to : " + str(train_vec.shape[1]) + "\n")

With the TFIDF leveraged, the vocabulary size shrinks to : 19681



In [231]:
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

#Model with no preprocessing 
tfidf_LR_model = LogisticRegression(penalty="l2", C = 100)
tfidf_LR_model.fit(train_vec, df_m2_train_label)

#makes predictions on the dev dataset with the model I just built
tfidf_dev_pred_labels = tfidf_LR_model.predict(test_vec)
tfidf_dev_pred_probs = tfidf_LR_model.predict_proba(test_vec)

C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
35 min run time

In [232]:
from sklearn import metrics
metrics.f1_score(df_m2_test_label, tfidf_dev_pred_labels,average='weighted')

C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.002090443490307989

# BoW Binary

In [304]:
#reformatting the data for BoW model
method3_test_data = [] 
method3_test_label = [] 
for passage, question, answer in dict_test.values():
    method3_test_data.append(' '.join(passage + question))
    method3_test_label.append(' '.join(answer))
df_m3_test_data = pd.DataFrame(method3_test_data)    
df_m3_test_label = pd.DataFrame(method3_test_label)

method3_train_data = [] 
method3_train_label = [] 
for passage, question, answer in dict_train.values():
    method3_train_data.append(' '.join(passage + question))
    method3_train_label.append(' '.join(answer))
df_m3_train_data = pd.DataFrame(method3_train_data)    
df_m3_train_label = pd.DataFrame(method3_train_label)

df_m3_train_data.fillna("", inplace=True)  
df_m3_test_data.fillna("", inplace=True) 

In [277]:
answers_train

array([778723, 778744,   7878, ..., 467242, 778100, 778785])

In [278]:
answers_test

array([778726, 778656, 769663, ..., 778748, 745129, 776480])

In [299]:
m3_test_labels = []
for item in answers_test:
    m3_test_labels.append(vocab[item-1])

In [288]:
len(m3_train_labels)

3048

In [296]:
m3_train_labels = []
for item in answers_train:
    m3_train_labels.append(vocab[item-1])

778723
778744
7878
778766
778776
778761
778739
763102
378
627180
777968
778744
7874
778744
776946
706367
778740
778785
778761
778652
778663
778785
7878
778785
778744
1104
773417
1181
778785
759098
778785
778806
442618
778723
778785
778748
778785
778785
729307
745116
778748
726270
775495
7878
775495
778766
7878
763525
7878
778785
778652
778785
778748
778766
778761
778761
7878
778748
763525
7874
778757
778785
747576
2749
778785
7878
763074
778744
778774
759523
745612
778744
7874
759523
760737
778774
778785
746570
3497
778580
778811
759523
778798
778785
3860
3887
778785
778637
778766
753073
778744
778744
661539
778744
778785
661539
778785
778774
778751
778766
776691
776751
778761
778785
778761
763218
5162
778702
778785
647391
778805
778744
759523
778785
388634
5133
778785
778785
7878
778785
7874
778807
778807
777671
778811
5534
778761
778776
778740
776793
7874
778761
778785
778766
778723
778785
769383
778748
761525
778785
712988
7878
778785
778744
7874
770669
6439
761363
778766
761525
778

778785
776805
778785
778763
778748
778748
778740
769082
762806
370322
778785
778342
778748
762479
778805
778748
778774
778774
778726
778785
776838
775095
778785
762806
778412
542114
718218
78081
78129
778753
765834
778785
778761
778807
669993
778748
778748
778766
778748
777970
778785
676677
745898
751180
732168
778796
778785
778748
639131
752214
778785
726714
706582
763240
771830
713635
346880
82452
751623
732710
752072
778785
79735
778785
778748
778785
79948
778734
728177
778785
778752
776145
778609
778776
778785
753566
777978
747322
80686
684714
751623
778703
762307
778785
778748
778801
778734
730426
778702
346880
753087
81364
778748
81446
778774
772925
763261
697118
755461
778748
778247
81805
778761
81875
778785
778785
82452
721005
775871
82143
743048
778726
778748
778748
778637
778785
778785
778748
82606
728446
778785
778785
82793
548757
778748
718100
87218
83084
83493
778785
87218
773076
633246
778748
778592
778775
723426
778748
86443
83996
759901
585194
772783
778748
753566
77878

778734
778766
778726
778753
778785
778785
767540
742483
778601
778806
778751
778748
778785
778785
778785
777956
778761
778222
778748
353102
778806
778798
778761
772628
353102
778785
778785
721476
778766
778785
776840
778222
778766
778748
778766
778752
778785
776124
155326
353102
778748
697677
778734
155638
778766
778766
155751
762734
778766
778785
778002
778504
778785
778748
778766
778766
778734
778785
778766
157430
778761
778726
778785
778744
778766
778785
778002
769355
778716
778748
778734
778174
778652
778785
777873
778785
645956
778785
778748
162288
714373
778785
778785
776356
772628
778210
778766
778785
778785
778638
778702
778785
778798
353102
387719
778761
778785
778002
778748
177220
778785
778785
778783
778748
778785
778026
778751
778806
778222
778734
778734
602855
773307
778734
748075
778766
778808
778766
778243
163238
778785
609765
778222
165420
778776
778744
774461
778785
772835
164274
165197
737659
775495
778774
763298
778734
164805
748075
778785
778783
165593
760685
165420

763218
778734
737959
233989
778325
778785
778785
778773
778761
778766
644110
778766
778806
778726
778637
336573
778776
763523
778748
763218
778785
778734
778785
778785
778726
708987
769005
233634
778766
778766
778807
763509
776946
778761
778785
778785
778785
778774
778785
778766
778766
778783
778785
776946
778761
778785
778761
778748
778380
762747
518991
760737
778766
778642
396968
778761
778785
778748
777968
763375
778726
778761
778761
235808
235880
778748
763126
776793
778761
778785
778748
778785
778712
757750
778785
778785
771140
760737
778785
763088
778785
763103
236819
763335
763509
762806
775495
756111
778785
778748
778652
237944
778785
778785
778785
778751
755647
774345
711951
237531
778785
778748
591719
778702
778776
778748
777575
627835
778734
778774
778748
778722
778748
778748
778726
778748
778716
778748
778785
778748
778703
778798
778702
778751
238564
778766
761282
543579
778761
778748
778785
778766
778637
667219
778776
778766
778785
778748
775601
778744
778785
758681
778726

778702
778785
778766
778785
778721
778785
549763
778785
778766
778785
778734
778748
778753
763673
778785
778806
778785
778609
778785
778783
778810
778740
778806
751668
778766
778785
778744
776946
778766
778766
778808
778712
774625
778785
778761
778783
778785
778761
778785
335433
778808
778260
778785
777283
778726
778761
778761
778761
778766
501733
778811
778761
778806
778785
778785
778744
778766
778748
778785
778785
778766
778785
573798
778785
778808
778785
778734
755461
778785
778744
778294
775979
778744
764738
778660
778785
778601
778785
778785
710340
734129
778748
607878
778785
778702
778748
778761
778807
726142
778247
778766
726010
778761
778805
778734
739788
778771
778761
778785
483829
778766
776513
778761
778740
778766
778766
777433
778565
720157
741504
778774
730426
778774
771137
776946
778766
730426
338940
778785
778766
778766
778243
778785
778776
778785
751466
629230
339375
752214
778783
752214
778785
778785
778798
778761
778204
778785
339759
751180
776751
755461
778785
778761

397026
778766
778785
778771
778785
778785
778785
778784
778734
534993
778771
778771
778726
778766
778761
778748
778753
778785
778750
778785
399320
778785
400480
399320
778771
778785
762479
778726
778763
778785
778785
401615
778734
535944
778785
778785
778785
778748
778744
778761
778785
778774
777374
778776
778785
778805
777886
778748
777940
778785
778785
778785
778806
778766
778798
778785
699893
778785
778785
778785
778748
778785
778703
778748
778785
778806
778774
778766
778776
778766
778761
778785
778785
399894
778776
778785
778785
762479
778785
778785
778785
761525
778761
778808
778748
778785
778785
778785
778748
778785
778766
778785
778766
778748
778703
778785
778785
778785
778748
740633
778761
778703
778761
778766
775379
778766
535944
778761
778774
778785
778748
778774
761525
778740
778748
778811
778740
778785
778748
778785
778761
778748
778785
778785
778785
778785
778785
701904
778748
778785
778774
778771
778766
736084
778785
754348
778785
778752
778740
778703
778785
778785
778785

753566
778785
778688
778785
778726
778761
778748
778703
778771
778748
778806
778785
778785
753934
778806
778726
778726
778748
777374
464041
748902
778766
778663
778785
778748
778748
778748
754070
778785
778748
778642
678979
778373
778785
778785
760340
778766
778776
778734
778805
778766
778761
778766
778761
778726
778761
600380
760340
778785
778785
778237
762796
778748
778751
778748
778734
778641
778247
778766
778785
778734
731270
778785
778703
778785
778785
778785
778734
778748
646195
600380
466865
778251
466853
777303
778785
778734
778761
778785
778734
778766
466865
778783
778785
762155
778766
778761
778734
778726
778785
778785
763509
778734
778761
778172
778758
778734
467242
778100
778785


In [297]:
len(m3_train_labels)

9870

In [298]:
m3_train_labels

['two',
 'of',
 'passepartout',
 'and',
 'on',
 'to',
 'here',
 'four',
 'barings',
 '840',
 'five',
 'of',
 'fogg',
 'of',
 'her',
 '700',
 'in',
 'the',
 'to',
 'by',
 'with',
 'the',
 'passepartout',
 'the',
 'of',
 'queenstown',
 'thousand',
 '770',
 'the',
 'hours',
 'the',
 'will',
 'speedy',
 'two',
 'the',
 'a',
 'the',
 'the',
 'chicago',
 'omaha',
 'a',
 'fort',
 'three',
 'passepartout',
 'three',
 'and',
 'passepartout',
 'colonel',
 'passepartout',
 'the',
 'by',
 'the',
 'a',
 'and',
 'to',
 'to',
 'passepartout',
 'a',
 'colonel',
 'fogg',
 'how',
 'the',
 'city',
 'ogden',
 'the',
 'passepartout',
 'seven',
 'of',
 'for',
 'fix',
 'mountains',
 'of',
 'fogg',
 'fix',
 'street',
 'for',
 'the',
 'francisco',
 '180th',
 'day',
 'friends',
 'fix',
 'who',
 'the',
 'yen',
 'batulcar',
 'the',
 'an',
 'and',
 'singing',
 'of',
 'of',
 'november',
 'of',
 'the',
 'november',
 'the',
 'for',
 'one',
 'and',
 'south',
 'north',
 'to',
 'the',
 'to',
 'john',
 'yokohama',
 'it',

In [305]:
df_m3_train_data = df_m3_train_data[0].astype(str)
print(df_m3_train_data.head()) #.astype(str)

df_m3_test_data = df_m3_test_data[0] #.astype(str)
print(df_m3_test_data.head())

0    you were ruined but now you are rich again 180...
1    the reader will remember that at five minutes ...
2    he deducted however from passepartouts share t...
3    passepartout went on his errand enchanted 177....
4    you have made a mistake of one day we arrived ...
Name: 0, dtype: object
0    is that the chance and hope you mentioned jaco...
1    marleys ghost marley was dead to begin with 2....
2    why did i walk through crowds of fellow beings...
3    mind i dont mean to say that i know of my own ...
4    you wish to be anonymous i wish to be left alo...
Name: 0, dtype: object


In [306]:
# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import * 
##################################################
vectorizer = TfidfVectorizer(lowercase=False) #already converted to lower case earlier, no need to run 2x
train_vec = vectorizer.fit_transform(df_m3_train_data)
test_vec = vectorizer.transform(df_m3_test_data)

print ("With the TFIDF leveraged, the vocabulary size shrinks to : " + str(train_vec.shape[1]) + "\n")

With the TFIDF leveraged, the vocabulary size shrinks to : 20730



In [307]:
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

start_time = time.time()

#Model with no preprocessing 
tfidf_LR_model_m1 = LogisticRegression(penalty="l2", C = 100)
tfidf_LR_model_m1.fit(train_vec, m3_train_labels)

#makes predictions on the dev dataset with the model I just built
tfidf_dev_pred_labels_m1 = tfidf_LR_model_m1.predict(test_vec)
tfidf_dev_pred_probs_m1 = tfidf_LR_model_m1.predict_proba(test_vec)

from sklearn import metrics
print(metrics.f1_score(m3_test_labels, tfidf_dev_pred_labels_m1, average='weighted'))

end_time = time.time()
run_time = time.strftime("%H:%M:%S",time.gmtime(end_time - start_time))
print ('Model run time (H:M:S) ' + run_time)

0.07346241812845043
Model run time (H:M:S) 00:05:44


C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\lucky\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
